# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [49]:
import os
#Disable gpu and enable cpu only
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
import random
import math
import numpy as np
import scipy.misc
import matplotlib
import matplotlib.pyplot as plt
import cv2

import coco
import utils
import model as modellib
import visualize

import pickle
from tqdm import tqdm
import operator


%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to trained weights file
# Download this file and place in the root of your 
# project (See README file for details)
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [2]:
def show_image(image):
    height = image.shape[0]
    width = image.shape[1]
    
    margin=50 # pixels
    dpi=100. # dots per inch

    figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
    left = margin/dpi/figsize[0] #axes ratio
    bottom = margin/dpi/figsize[1]

    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)

    plt.imshow(image)
    plt.show()

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [3]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.print()


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.002
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 123.7  116.8  103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINI

## Create Model and Load Trained Weights

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [5]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [6]:
with open('../refer/my_refer_data.pickle', 'rb') as file:
    my_refer_data = pickle.load(file)

In [7]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    # return the intersection over union value
    return iou

## Run Object Detection

In [ ]:
pairs = dict()

for ref_id in tqdm(my_refer_data['ref_ids']):
    x,y,w,h = my_refer_data['boxes'][ref_id]
    x1,y1,x2,y2 = x,y,x+w,y+h
    
    # Load a random image from the images folder
    file_names = next(os.walk(IMAGE_DIR))[2]
    #image = scipy.misc.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
    image = scipy.misc.imread(my_refer_data['image_file_names'][ref_id])
    if len(image.shape) != 3:
        continue
    
    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results
#     r = results[0]
#     visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
#                                 class_names, r['scores'])
    
    class_ids = results[0]['class_ids']
    boxes = results[0]['rois']
    if len(boxes) == 0:
        continue
    
    box = boxes[0]
    bx1,by1,bx2,by2 = box[1],box[0],box[3],box[2]
    max_iou = bb_intersection_over_union((x1,y1,x2,y2), (bx1,by1,bx2,by2))
    
    max_iou_box_index = 0
    max_iou_box = bx1,by1,bx2,by2
    
    for i, box in enumerate(boxes):
        bx1,by1,bx2,by2 = box[1],box[0],box[3],box[2]
        iou = bb_intersection_over_union((x1,y1,x2,y2), (bx1,by1,bx2,by2))
        if iou > max_iou:
            max_iou = iou
            max_iou_box_index = i
            max_iou_box = bx1,by1,bx2,by2
    
    refer_subject = my_refer_data['subjects'][ref_id]
    mask_rcnn_class = class_names[class_ids[max_iou_box_index]]
    
#     print(my_refer_data['subjects'][ref_id], class_names[class_ids[max_iou_box_index]])
    
    if refer_subject not in pairs:
        pairs[refer_subject] = dict()
    pair = pairs[refer_subject]
    if mask_rcnn_class in pair:
        pair[mask_rcnn_class] += 1
    else:
        pair[mask_rcnn_class] = 1
    
#     image = scipy.misc.imread(my_refer_data['image_file_names'][ref_id])
#     cv2.rectangle(image,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0),3)
#     cv2.rectangle(image,(max_iou_box[0],max_iou_box[1]),(max_iou_box[2],max_iou_box[3]),(255,0,0),3)
    
#     print('max_iou',max_iou)
#     show_image(image)
    

100%|██████████| 40062/40062 [3:43:58<00:00,  2.98it/s]  


In [46]:
# with open('pairs.pickle', 'wb') as file:
#     pickle.dump(pairs, file)

In [45]:
with open('pairs.pickle', 'rb') as file:
    pairs = pickle.load(file)

In [55]:
best_pairs = dict()
for pair in pairs:
    best_pairs[pair] = max(pairs[pair].items(), key=operator.itemgetter(1))[0]